In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://uacparts.com/Catalog/Buyers?part=' + input_.loc[a, 'UAC_Part_Number'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.strip(), 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                list_tr = html.xpath('//table/tr')
                for tr in list_tr:
                    list_td = tr.xpath('./td')
                    make = '' if list_td[1].text == None else list_td[1].text.strip()
                    model = '' if list_td[2].text == None else list_td[2].text.strip()
                    engine = '' if list_td[4].text == None else list_td[4].text.strip()
                    year = int(list_td[0].text.strip())

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: {engine: [year]}}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = {engine: [year]}
                        else:
                            if engine not in dict_vehicle[make][model]:
                                dict_vehicle[make][model][engine] = [year]
                            else:
                                dict_vehicle[make][model][engine] += [year]

                    dict_vehicle[make][model][engine] = sorted(list(set(dict_vehicle[make][model][engine])))

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_year = []
                        for engine in dict_vehicle[make][model]:
                            list_year += dict_vehicle[make][model][engine]
                        list_year = sorted(list(set(list_year)))

                        year_min = min(list_year)
                        year_max = max(list_year)
                        if year_min == year_max:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                vehicle_1 = '\n'.join(sorted(list_vehicle))

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        for engine in dict_vehicle[make][model]:
                            year_min = min(dict_vehicle[make][model][engine])
                            year_max = max(dict_vehicle[make][model][engine])
                            if year_min == year_max:
                                if engine == '':
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                else:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                            else:
                                if engine == '':
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                else:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                vehicle_2 = '\n'.join(sorted(list_vehicle))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'UAC_Part_Number': input_.loc[a, 'UAC_Part_Number'],
                                         'UAC_Vehicle_1': vehicle_1,
                                         'UAC_Vehicle_2': vehicle_2}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'UAC_Part_Number': input_.loc[a, 'UAC_Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'UAC_Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['UAC_Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./uac_vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['UAC_Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./uac_vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：300

[ok] - CO 0040C
[尝试次数：1] - [剩余数量：280] - [当前时间：19:44:49]

[ok] - CO 0042C
[尝试次数：1] - [剩余数量：279] - [当前时间：19:44:49]

[ok] - CO 0043C
[尝试次数：1] - [剩余数量：278] - [当前时间：19:44:49]

[ok] - CO 11463V
[尝试次数：1] - [剩余数量：277] - [当前时间：19:44:49]

[ok] - CO 0041C
[尝试次数：1] - [剩余数量：276] - [当前时间：19:44:49]

[ok] - CO 11136N
[尝试次数：1] - [剩余数量：275] - [当前时间：19:44:49]

[ok] - CO 11460V
[尝试次数：1] - [剩余数量：274] - [当前时间：19:44:49]

[ok] - CO 1100JC
[尝试次数：1] - [剩余数量：273] - [当前时间：19:44:49]

[ok] - CO 11444V
[尝试次数：1] - [剩余数量：272] - [当前时间：19:44:50]

[ok] - CO 0024GLC
[尝试次数：1] - [剩余数量：271] - [当前时间：19:44:51]

[ok] - CO 0071C
[尝试次数：1] - [剩余数量：270] - [当前时间：19:44:51]

[ok] - CO 0073C
[尝试次数：1] - [剩余数量：269] - [当前时间：19:44:51]

[ok] - CO 0076C
[尝试次数：1] - [剩余数量：268] - [当前时间：19:44:51]

[ok] - CO 04080C
[尝试次数：1] - [剩余数量：267] - [当前时间：19:44:51]

[ok] - CO 10541AC
[尝试次数：1] - [剩余数量：266] - [当前时间：19:44:51]

[ok] - CO 11513C
[尝试次数：1] - [剩余数量：265] - [当前时间：19:44:51]

[ok] - CO 11282C
[尝试次数：1] - [剩余数量：264] - [当前时间：19:44:51]

[ok] - CO 

[ok] - CO 46234Q
[尝试次数：1] - [剩余数量：137] - [当前时间：19:45:51]

[ok] - CO 4418
[尝试次数：19] - [剩余数量：136] - [当前时间：19:45:51]

[ok] - CO 4630DKV
[尝试次数：1] - [剩余数量：135] - [当前时间：19:45:52]

[ok] - CO 4623C
[尝试次数：1] - [剩余数量：134] - [当前时间：19:45:52]

[ok] - CO 4634DKQ
[尝试次数：1] - [剩余数量：133] - [当前时间：19:45:52]

[ok] - CO 4622C
[尝试次数：1] - [剩余数量：132] - [当前时间：19:45:52]

[ok] - CO 4637C
[尝试次数：1] - [剩余数量：131] - [当前时间：19:45:52]

[ok] - CO 4634DKV
[尝试次数：1] - [剩余数量：130] - [当前时间：19:45:52]

[ok] - CO 4630DKQ
[尝试次数：1] - [剩余数量：129] - [当前时间：19:45:53]

[ok] - CO 4643
[尝试次数：1] - [剩余数量：128] - [当前时间：19:45:53]

[ok] - CO 4646C
[尝试次数：1] - [剩余数量：127] - [当前时间：19:45:53]

[ok] - CO 4645C
[尝试次数：1] - [剩余数量：126] - [当前时间：19:45:53]

[ok] - CO 4643C
[尝试次数：1] - [剩余数量：125] - [当前时间：19:45:53]

[ok] - CO 4615DKV
[尝试次数：1] - [剩余数量：124] - [当前时间：19:45:53]

[ok] - CO 4647
[尝试次数：1] - [剩余数量：123] - [当前时间：19:45:54]

[ok] - CO 4326DC
[尝试次数：40] - [剩余数量：122] - [当前时间：19:45:54]

[ok] - CO 46540JC
[尝试次数：1] - [剩余数量：121] - [当前时间：19:45:54]

[ok] - CO 4652C
[尝

[ok] - CO 4663C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:46:34]

[ok] - CO 7819C
[尝试次数：1] - [剩余数量：0] - [当前时间：19:46:38]

[ok] - CO 4816C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:46:39]

[ok] - CO 4848C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:46:42]

[ok] - CO 6130DC
[尝试次数：3] - [剩余数量：0] - [当前时间：19:46:44]

[ok] - CO 8044C
[尝试次数：8] - [剩余数量：0] - [当前时间：19:46:48]

[ok] - CO 5072C
[尝试次数：9] - [剩余数量：0] - [当前时间：19:46:51]

[ok] - CO 8203C
[尝试次数：7] - [剩余数量：0] - [当前时间：19:46:59]

[ok] - CO 7312C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:47:00]

[ok] - CO 8148C
[尝试次数：5] - [剩余数量：0] - [当前时间：19:47:05]

[ok] - CO 6321
[尝试次数：7] - [剩余数量：0] - [当前时间：19:47:22]

输出ing...

Done ~
